# Init Database

In [5]:
import pandas as pd
import numpy as np
from tqdm.notebook import trange, tqdm
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
load_dotenv()

True

## Load datasource

In [6]:
# load "../wine_training.csv" as source_data in pandas dataframe
# csv is delimeted with semicolon
# first row is the header and first column is the index
source_data = pd.read_csv("../wine_training.csv", sep=";", header=0, index_col=0)

source_data.head(10)

,wine type,fixed acidity,volatile acidity,citric acid,residual sugar,magnesium,flavanoids,minerals,calcium,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,Pinot noir,5.8,0.15,0.49,1.1,76.729301,894.94,186.639301,109.91,0.048,21.0,98.0,0.99290,3.19,0.48,9.2,5
1,Merlot,6.6,0.25,0.32,5.6,4.795712,1160.95,251.875712,247.08,0.039,15.0,68.0,0.99163,2.96,0.52,11.1,6
2,Chardonnay,6.7,0.21,0.34,1.5,85.193710,789.82,304.703710,219.51,0.035,45.0,123.0,0.98949,3.24,0.36,12.6,7
3,Merlot,8.3,0.28,0.27,17.5,11.976525,777.86,237.586525,225.61,0.045,48.0,253.0,1.00014,3.02,0.56,9.1,6
4,Merlot,7.5,0.42,0.19,6.9,5.599673,785.72,95.399673,89.80,0.041,62.0,150.0,0.99508,3.23,0.37,10.0,6
5,Merlot,7.3,0.34,0.30,1.3,22.403749,1044.95,289.523749,267.12,0.057,25.0,173.0,0.99480,3.26,0.51,9.1,6
6,Merlot,7.6,0.21,0.49,2.5,23.875866,888.61,133.545866,109.67,0.047,20.0,130.0,0.99178,3.15,0.48,11.1,5
7,Chardonnay,6.0,0.25,0.40,5.7,23.309699,1381.79,266.529699,243.22,0.052,56.0,152.0,0.99398,3.16,0.88,10.5,6
8,Cabernet Sauvignon,6.7,0.18,0.19,4.7,49.165745,1456.41,269.915745,220.75,0.046,57.0,161.0,0.99460,3.32,0.66,10.5,6
9,Gamay,7.7,0.28,0.39,8.9,54.450579,929.44,377.690579,323.24,0.036,8.0,117.0,0.99350,3.06,0.38,12.0,2


In [7]:
source_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   wine type             4000 non-null   object 
 1   fixed acidity         4000 non-null   float64
 2   volatile acidity      4000 non-null   float64
 3   citric acid           3997 non-null   float64
 4   residual sugar        3996 non-null   float64
 5   magnesium             3996 non-null   float64
 6   flavanoids            3996 non-null   float64
 7   minerals              3996 non-null   float64
 8   calcium               3996 non-null   float64
 9   chlorides             3996 non-null   float64
 10  free sulfur dioxide   3996 non-null   float64
 11  total sulfur dioxide  3996 non-null   float64
 12  density               4000 non-null   float64
 13  pH                    4000 non-null   float64
 14  sulphates             4000 non-null   float64
 15  alcohol              

## Minimal data preparation for the database

In [8]:
# column wine type is an object
# convert it to category
source_data["wine type"] = source_data["wine type"].astype("category")
source_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   wine type             4000 non-null   category
 1   fixed acidity         4000 non-null   float64 
 2   volatile acidity      4000 non-null   float64 
 3   citric acid           3997 non-null   float64 
 4   residual sugar        3996 non-null   float64 
 5   magnesium             3996 non-null   float64 
 6   flavanoids            3996 non-null   float64 
 7   minerals              3996 non-null   float64 
 8   calcium               3996 non-null   float64 
 9   chlorides             3996 non-null   float64 
 10  free sulfur dioxide   3996 non-null   float64 
 11  total sulfur dioxide  3996 non-null   float64 
 12  density               4000 non-null   float64 
 13  pH                    4000 non-null   float64 
 14  sulphates             4000 non-null   float64 
 15  alco

## Data Upload

In [9]:
mysql_settings = {
    "host": os.environ.get("DB_HOST"),
    "user": os.environ.get("DB_USER"),
    "password": os.environ.get("DB_PASSWORD"),
    "database": os.environ.get("DB_NAME")
}
database_engine = create_engine("mysql+mysqldb://{}:{}@{}/{}".format(
    mysql_settings["user"],
    mysql_settings["password"],
    mysql_settings["host"],
    mysql_settings["database"]),  pool_size=20, max_overflow=0)

try:
    database_engine.connect()
    print("Connection to database established")
except:
    print("Connection to database failed")


Connection to database established


In [10]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))
table_name = "wines"

df = source_data.copy()
df = df.reset_index(drop=True)
chunksize = 100
amount_data = 0
with tqdm(total=len(df), desc="Adding data to {}".format(table_name)) as pbar:
    for i, cdf in enumerate(chunker(df, chunksize)):
        cdf.to_sql(name=table_name, con=database_engine, if_exists="append", index=False)
        pbar.update(chunksize)
        tqdm._instances.clear()

Adding data to wines:   0%|          | 0/4000 [00:00<?, ?it/s]